In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append("../../functions")

from Utils import *
from Plotting import *
from Fitting import fit_for_all_subjects_parallel, fit_brier
from Forward import forward_fitting
from Backward import fit_model_backward


In [3]:
chance_level_subjects = [45634, 46069, 46279, 46303, 46651, 47398]
data = preprocess_data("../../data/twoArmedandit_with_confidence.csv", chance_level_subjects, "induction", "lc1")
subjects = data["sub"].unique()
n_fitting = 120
n_threads = 100

In [5]:
forward_params_ranges = [[0.01, 1], [0.01, 100], [1, 5], [0.1, 1]]
forward_res = fit_for_all_subjects_parallel(forward_params_ranges, n_threads, n_fitting, subjects, data, forward_fitting, True)
forward_df = make_df_after_fitting(forward_res, ["alpha", "beta", "lbound", "bound_range"], subjects)
forward_df["hbound"] = (5 - forward_df["lbound"]) * forward_df["bound_range"] + forward_df["lbound"]
forward_df.to_csv("forward_df_high.csv")


In [6]:
backward_params_ranges = [[0.01, 1], [0.01, 100], [1, 5], [0.1, 1]]
backward_res = fit_for_all_subjects_parallel(backward_params_ranges, n_threads, n_fitting, subjects, data, fit_model_backward, True)
backward_df = make_df_after_fitting(backward_res, ["alpha", "beta", "lbound", "bound_range"], subjects)
backward_df["hbound"] = (5 - backward_df["lbound"]) * backward_df["bound_range"] + backward_df["lbound"]
backward_df.to_csv("backward_df_high.csv")

In [7]:
backward_params_ranges_unscaled = [[0, 1], [0, 100]]
backward_res_unscaled = fit_for_all_subjects_parallel(backward_params_ranges_unscaled, n_threads, n_fitting, subjects, data, fit_model_backward, False)
backward_df_unscaled = make_df_after_fitting(backward_res_unscaled, ["alpha", "beta"], subjects)
backward_df_unscaled["lbound"], backward_df_unscaled["hbound"] = 1, 5
backward_df_unscaled.to_csv("backward_df_unscaled_high.csv")

In [8]:
from Fitting import fit_brier

brier_res = fit_brier(subjects, data, n_threads=n_threads)
scaled_brier_ls = []
scaled_brier_lb = []
scaled_brier_ub = []
for res in brier_res:
    scaled_brier_ls.append(res[1])
    scaled_brier_lb.append(res[0][0])
    scaled_brier_ub.append(res[0][1] * (1 - res[0][0]) + res[0][0])
scaled_brier_df = pd.DataFrame(
    list(zip(subjects, scaled_brier_lb, scaled_brier_ub, scaled_brier_ls)),
    columns=["subject", "lb", "ub", "brier"],
)
scaled_brier_df["scaled_QSR"] = 1 - scaled_brier_df["brier"]
scaled_brier_df.to_csv("scaled_brier_df_high.csv")

In [1]:
!pip install numba


ERROR: Could not find a version that satisfies the requirement numba
ERROR: No matching distribution found for numba
